### Handle ERA5 dataset with Zampy
Demo notebook for developers.

In [1]:
import numpy as np
from zampy.datasets import ERA5
from zampy.datasets.dataset_protocol import TimeBounds, SpatialBounds
from pathlib import Path

In [2]:
work_dir = Path("/home/yangliu/EcoExtreML/temp")
download_dir = work_dir / "download"
ingest_dir = work_dir / "ingest"
times = TimeBounds(np.datetime64("2010-01-01T00:00:00"), np.datetime64("2010-01-31T23:00:00"))
bbox_demo = SpatialBounds(54, 56, 1, 3)

Download dataset.

In [3]:
era5_dataset = ERA5()
era5_dataset.download(
    download_dir=download_dir,
    time_bounds=times,
    spatial_bounds=bbox_demo,
    variable_names=["10m_v_component_of_wind"], #"surface_pressure", "mean_total_precipitation_rate"
)

File 'era5_10m_v_component_of_wind_2010-1.nc' already exists, skipping...


True

Data ingestion to the unified format in `zampy`.

In [4]:
era5_dataset.ingest(download_dir, ingest_dir)

File 'era5_10m_v_component_of_wind_2010-1.nc' already exists, skipping...


True

In [5]:
ds = era5_dataset.load(
    ingest_dir=ingest_dir,
    time_bounds=times,
    spatial_bounds=bbox_demo,
    variable_names=["10m_v_component_of_wind"],
    resolution=1.0,
    regrid_method="flox",
)

In [6]:
ds

<xarray.Dataset>
Dimensions:                  (time: 744, latitude: 54, longitude: 54)
Coordinates:
  * time                     (time) datetime64[ns] 2010-01-01 ... 2010-01-31T...
  * latitude                 (latitude) float64 1.0 2.0 3.0 ... 52.0 53.0 54.0
  * longitude                (longitude) float64 3.0 4.0 5.0 ... 54.0 55.0 56.0
Data variables:
    10m_v_component_of_wind  (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 54), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-07-11 10:17:10 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [7]:
from zampy.datasets import converter

ds_convert = converter.convert(ds, era5_dataset, "ALMA")

10m_v_component_of_wind renamed to Wind_E.
Conversion of dataset 'era5' following ALMA convention is complete!


In [8]:
ds_convert

<xarray.Dataset>
Dimensions:    (time: 744, latitude: 54, longitude: 54)
Coordinates:
  * time       (time) datetime64[ns] 2010-01-01 ... 2010-01-31T23:00:00
  * latitude   (latitude) float64 1.0 2.0 3.0 4.0 5.0 ... 51.0 52.0 53.0 54.0
  * longitude  (longitude) float64 3.0 4.0 5.0 6.0 7.0 ... 53.0 54.0 55.0 56.0
Data variables:
    Wind_E     (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 54), meta=np.ndarray>
Attributes:
    Conventions:  ALMA
    history:      2023-07-11 10:17:10 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...